In [1]:
import pandas as pd
import numpy as np

df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]})

In [2]:
df

,A,B,C
0,1,4,7
1,2,5,8
2,3,6,9


In [4]:
df.apply(lambda row:pd.DataFrame(row), axis = 1)

0       0
A  1
B  4
C  7
1       1
A  2
B  5
C  8
2       2
A  3
B  6
C  9
dtype: object

In [16]:
df.apply(lambda row:row['A'], axis = 1)

0    1
1    2
2    3
dtype: int64

In [5]:
df.apply(lambda row:pd.DataFrame([row]), axis = 1)

0       A  B  C
0  1  4  7
1       A  B  C
1  2  5  8
2       A  B  C
2  3  6  9
dtype: object

In [7]:
type(df['A'])

pandas.core.series.Series

In [10]:
type(df.iloc[0,:])

pandas.core.series.Series

In [12]:
slice = df.iloc[0,:]
slice

A    1
B    4
C    7
Name: 0, dtype: int64

In [13]:
slice['A']

1

In [18]:
from pyspark.sql import SparkSession
import pandas as pd

# 初始化 Spark 会话
spark = SparkSession.builder \
    .appName("ArrowOptimization") \
    .getOrCreate()

# 启用 Arrow 优化
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

# 创建 Pandas DataFrame
pandas_df = pd.DataFrame({
    'A': [1, 2, 3],
    'B': ['x', 'y', 'z'],
    'C': [4.0, 5.5, 6.1]
})

# 转换为 Spark DataFrame
spark_df = spark.createDataFrame(pandas_df)

# 转换回 Pandas DataFrame
result_pandas_df = spark_df.toPandas()

print(result_pandas_df)


   A  B    C
0  1  x  4.0
1  2  y  5.5
2  3  z  6.1


In [1]:
!pip freeze > requirements.txt

In [ ]:
import pandas as pd
import multiprocessing
from tqdm import tqdm

# 处理每个分组的函数
def process_group(group_sorted):
    selected_dates = []
    if len(group_sorted) == 1:
        selected_dates.append(group_sorted)
    elif len(group_sorted) == 2:
        date_diff = (group_sorted.iloc[1]['mdl_dte'] - group_sorted.iloc[0]['mdl_dte']).days
        if date_diff >= 30:
            selected_dates.append(group_sorted)
        else:
            selected_dates.append(group_sorted.iloc[:1])
    else:
        selected_dates.append(group_sorted.iloc[0])  # 选中第一个日期
        last_date = group_sorted.iloc[0]['mdl_dte']
        
        for i in range(1, len(group_sorted)):
            date_diff = (group_sorted.iloc[i]['mdl_dte'] - last_date).days
            if date_diff >= 30:
                selected_dates.append(group_sorted.iloc[i:i+1])
                last_date = group_sorted.iloc[i]['mdl_dte']
    return pd.DataFrame(selected_dates)

# 处理每个uid的函数
def process_repayment_data_parallel(df):
    # 按uid分组
    grouped = df.groupby('uid')

    # 结果存储
    results = []

    # 使用multiprocessing的Pool
    with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
        # 使用tqdm显示进度条
        results = list(tqdm(pool.imap(process_group, [group.sort_values(by='mdl_dte') for _, group in grouped]),
                            total=len(grouped), desc="Processing UIDs"))
    
    # 合并所有结果
    # 将所有非空的DataFrame合并
    valid_results = [result for result in results if not result.empty]
    
    if valid_results:
        return pd.concat(valid_results, ignore_index=True)
    else:
        print("No valid results to concatenate.")
        return pd.DataFrame()

# 示例数据
data = {
    'uid': ['A', 'A', 'A', 'B', 'B', 'C'],
    'mdl_dte': ['2024-01-01', '2024-02-15', '2024-03-30', '2024-01-01', '2024-02-01', '2024-01-01']
}
df = pd.DataFrame(data)
df['mdl_dte'] = pd.to_datetime(df['mdl_dte'])

# 执行并行处理
processed_df = process_repayment_data_parallel(df)
print(processed_df)

Processing UIDs:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
import multiprocessing
import time
import operator

def do_something(item):
    functions = [fun1, fun2, fun3, fun4]
    #print(item)
    #print(functions[item])
    functions[item]( )

def fun1():
    list(range(10, 300000000))

def fun2():
    list(range(10, 300000000))
    
def fun3():
    list(range(10, 300000000))

def fun4():
    list(range(10, 300000000))
    
if __name__ == '__main__':
    #### 多线程运行  ####
    start = time.time()
    items = [0,1,2,3]
    p = multiprocessing.Pool()
    b = p.map(do_something, items)
    p.close()
    p.join()
    end = time.time()
    print('multi processing time:',str(end-start),'s')
    #### 单线程运行 ####
    begin = time.time()
    functions = [fun1, fun2, fun3, fun4]
    for func in functions:
        func()
    end = time.time()
    process_time = end - begin
    print('single process time is:%s'%(str(process_time)))

In [1]:
def int_to_excel_column(n):
    result = ""
    while n > 0:
        n -= 1  # 调整n，使其从0开始，这样'Z'对应的值是26而不是25
        result = chr(n % 26 + 65) + result  # 将当前余数转换为字符
        n //= 26  # 减少n，进行下一轮的计算
    return result

# 示例
print(int_to_excel_column(1))   # 输出 "A"
print(int_to_excel_column(26))  # 输出 "Z"
print(int_to_excel_column(27))  # 输出 "AA"
print(int_to_excel_column(52))  # 输出 "AZ"
print(int_to_excel_column(53))  # 输出 "BA"

A
Z
AA
AZ
BA


In [1]:
import pandas as pd
import numpy as np
df = pd.DataFrame([[4, 9]] * 3, columns=['A', 'B'])
df

,A,B
0,4,9
1,4,9
2,4,9


In [3]:
df.apply(np.sum, axis = 'index')

A    12
B    27
dtype: int64

In [5]:
df.apply(np.sum, axis = 0)

A    12
B    27
dtype: int64

In [4]:
df.apply(np.sum, axis = 'columns')

0    13
1    13
2    13
dtype: int64

In [2]:
import numpy as np
x = np.array([1, 2, 3], np.int32)
type(x)

numpy.ndarray

In [3]:
x

array([1, 2, 3])

In [1]:
def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n ** 0.5) + 1):
        if n % i == 0:
            return False
    return True

# import multiprocessing
# multiprocessing.cpu_count()

In [2]:
num_list = [i + 10000 for i in range(10000)]
import time
start_time = time.time()
results = [is_prime(num) for num in num_list]
end_time = time.time()
print("Time taken for single thread: ", end_time - start_time)

Time taken for single thread:  0.006018638610839844


In [ ]:
import multiprocessing
multiprocessing.cpu_count()

def parallel_process(num_list):
    cpu_num = 5
    # num_list = [i + 10000 for i in range(10000)]
    with multiprocessing.Pool(cpu_num) as p:
        results = p.map(is_prime, num_list)
    return results

num_list = [i + 10000 for i in range(10000)]
parallel_process(num_list)

In [1]:
import concurrent.futures

def worker(num):
    print(f"Worker {num} is working")

if __name__ == "__main__":
    with concurrent.futures.ProcessPoolExecutor() as executor:
        executor.map(worker, range(10))
